In [33]:
import os
import json
import uuid
import pytz

from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import identify_cash_items
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response
)

from datetime import datetime
from collections import namedtuple
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
import lusid
import lusid.models as models
import pandas as pd

from lusidjam import RefreshingToken

LUSID_INSTRUMENT_IDENTIFIER = "Instrument/default/LusidInstrumentId"

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Set up API's

In [34]:
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
derived_portfolios = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
derived_transaction_portfolios_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)

# Create Portfolio and Properties

In [35]:
scope = "FourSided"
code = "UKBondPortfolio"
effective_date = "2021-01-01"

In [36]:
try:

    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=code,
            code=code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[f"Transaction/{scope}/CashType"],
        ),
    )
    
except lusid.ApiException as e:
    print(e.body)

In [37]:
properties = [
    ("CashType", "string"),
    ("PortBaseCurrency", "string"),
    ("PortBaseCurrencyIns", "string"),
    ("AccruedInterest", "number")
]

In [38]:
for property_code, dtype in properties:

    try:

        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code=dtype, scope="system"),
            )
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Error creating Property Definition 'Transaction/FourSided/CashType' because it already exists.
Error creating Property Definition 'Transaction/FourSided/PortBaseCurrency' because it already exists.
Error creating Property Definition 'Transaction/FourSided/PortBaseCurrencyIns' because it already exists.
Error creating Property Definition 'Transaction/FourSided/AccruedInterest' because it already exists.


# Upload the instruments/transactions file

In [45]:
holdings_df = pd.read_csv('data/fstt_holdings.csv')

In [46]:
holdings_df

,portfolio_code,portfolio_name,transaction_id,transaction_type,transaction_date,settlement_date,ISIN,name,units,price,amount_cost,exchange rate,AccruedInterest,PortBaseCurrency,PortBaseCurrencyIns,CashType,Unnamed: 16,Unnamed: 17
0,UKBondPortfolio,UK Bond Portfolio,TX001,FundsIn,01/01/2021,01/01/2021,CCY_GBP,Cash GBP,"1,000,000",1.00,"1,000,000.00",1.37,NaN,USD,GBP,RegCash,NaN,NaN
1,UKBondPortfolio,UK Bond Portfolio,TX002,fourSidedBuy,01/02/2021,04/02/2021,XS1888180996,VOD 4.875 10/03/78,"980,392",1.02,"1,000,000.00",1.36,24842.47,USD,GBP,NotCash,NaN,NaN
2,UKBondPortfolio,UK Bond Portfolio,TX003,fourSidedBuy,01/02/2021,04/02/2021,GB00BYZW3G56,UKT 1 1/2 07/22/26,"970,873",1.03,"1,000,000.00",1.36,3066.30,USD,GBP,NotCash,NaN,NaN
3,UKBondPortfolio,UK Bond Portfolio,TX004,fourSidedBuy,01/02/2021,04/02/2021,GB00BJLR0J16,UKT 1 5/8 10/22/54,"968,992",1.03,"1,000,000.00",1.36,7410.71,USD,GBP,NotCash,NaN,NaN


In [41]:
# upsert the instruments
instrument_identifier_mapping = {
    "ClientInternal": "ISIN",
}

instrument_mapping_required = {"name": "name"}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=holdings_df,
    mapping_required=instrument_mapping_required,
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,4,0,0


In [47]:
# map the transactions
identifiers = {
    "ClientInternal": "ISIN",
}

transaction_field_mapping = {
    "code": "portfolio_code",
    "transaction_id": "transaction_id",
    "type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "units",
    "transaction_price.price": "price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "amount_cost",
    "total_consideration.currency": "PortBaseCurrencyIns",
    "exchange_rate": "exchange rate",
    "transaction_currency": "PortBaseCurrencyIns",
}

In [48]:
# upsert transactions
responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=holdings_df,
    mapping_required=transaction_field_mapping,
    mapping_optional={},
    identifier_mapping=identifiers,
    file_type="transaction",
    property_columns=[
        "AccruedInterest",
        "PortBaseCurrency",
        "PortBaseCurrencyIns",
        "exchange rate",
        "CashType",
    ],
)

succ, failed = format_transactions_response(responses)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


In [49]:
# create the 4 sides required
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)

side_list = [
        models.SideConfigurationDataRequest(
            side="AccruedInterest3",
            security=f"Transaction/{scope}/PortBaseCurrencyIns",
            currency=f"Transaction/{scope}/PortBaseCurrency",
            rate="SettledToPortfolioRate",
            units=f"Transaction/{scope}/AccruedInterest",
            amount=f"Transaction/{scope}/AccruedInterest"
        )
    ]

for side in side_list:
    
    current_sides = [side.side for side in system_configuration_api.list_configuration_transaction_types().side_definitions]
    
    if side.side in list(current_sides):
        
        print(f"{side.side} already exists in LUSID")
    
    else:
        
        response = system_configuration_api.create_side_definition(side_configuration_data_request = side)
        
        print(f"{side.side} has been created in LUSID")

AccruedInterest3 already exists in LUSID


In [53]:
# create the transaction configuration
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="fourSidedBuy",
                description="An fourSidedBuy transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[
                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType",
                        set_to=f"NotCash")                  
                ]),
            
            
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[            
                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType",
                        set_to=f"RegCash")
                ]),
            
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="AccruedInterest3",
                direction=1,
                properties=None,
                mappings=[
                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType",
                        set_to=f"CashBucket2")
                    
                ]),
            
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="AccruedInterest3",
                direction=-1,
                properties=None,
                mappings=[
                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType",
                        set_to=f"CashBucket1")
                ]),
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)